# 運用系処理

## インデックス一覧
インデックス一覧を表示します。indexを指定しない場合、すべてのインデックスが表示されます。  
indexを指定した場合、そのパターンに一致するインデックス一覧が表示されます。  
例）ElasticManager.show_indices(index="rawdata-*")  
⇒ rawdataインデックスの一覧が表示されます。

In [5]:
import os, sys
sys.path.append(os.path.join(os.getcwd(), '../src/'))
from elastic_manager.elastic_manager import ElasticManager
import pandas as pd
pd.set_option('display.max_rows', 1000) # 最大表示数

ElasticManager.show_indices(index="rawdata-*")

,index,docs.count,store.size
0,rawdata-20201216165900,0,0
1,rawdata-20210118153002,156672,14177
2,rawdata-20210119104121,157696,14345
3,rawdata-20210119115701,6154240,556826
4,rawdata-20210119115715,0,0
5,rawdata-20210119115802,0,0
6,rawdata-20210120002716,6154240,555644
7,rawdata-20210120010638,1108992,99052
8,rawdata-20210120012626,315392,27902
9,rawdata-20210120014154,788480,70311


## インデックス削除
DELETE_INDEXに削除するインデックス名を入力します。ワイルドカード(\*)が使用可能です。  
例）rawdata-20201201\*  
⇒ 2020/12/01のrawdataデータインデックスがすべて削除されます。

In [11]:
DELETE_INDEX = 'rawdata-20201113-*'

In [12]:
from elastic_manager import ElasticManager

ElasticManager.delete_index(DELETE_INDEX)

rawdata-20201113-*を削除しました。


## データ削除（連番指定）
連番指定でデータを削除します。以下のパラメータを指定してください。

* TARGET_INDEX：データ削除対象のインデックス名  
* START_SEQ_NUM：開始番号  
* END_SEQ_NUM：終了番号

In [2]:
TARGET_INDEX = 'rawdata-20201112-1'
START_SEQ_NUM = 0
END_SEQ_NUM = 10

In [3]:
from elastic_manager import ElasticManager

ElasticManager.delete_data_by_seq_num(index=TARGET_INDEX, start=START_SEQ_NUM, end=END_SEQ_NUM)

データを11件削除しました。


## データ削除（ショット番号指定）
ショット番号指定でデータを削除します。以下のパラメータを指定してください。  

* TARGET_INDEX：データ削除対象のインデックス名  
* SHOT_NUMBER：ショット番号

In [ ]:
TARGET_INDEX = 'shots-20201112'
SHOT_NUMBER = 1

In [ ]:
from elastic_manager import ElasticManager

ElasticManager.delete_data_by_shot_num(index=TARGET_INDEX, shot_number=SHOT_NUMBER)